# Web Scraping in Python for Beginners

In this notebook, I will give an introduction to webscraping in python. This will not be an exhaustive overview of all the resources at your disposal with python, but it should be enough to help you in the beginning. The library I will be going over in this post is Beautiful Soup (utilizing Requests). In my next tutorial, I will go over how to utalize Beautiful Soup and Selenium together. The only other major option you have available is Scrapy. Along the way, I will discuss your options and leave more tutorials for different aspects of this project that are outside of the scope of Beautiful Soup.

#### First we need to import the needed libraries: 

In [2]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd
import re
import urllib

### Beautiful Soup _Only_:

Beautiful Soup is a python package that parces XML and HTML. It makes it very easy to get data from the html data avaible on the website. However, it does have it's pitfalls. These include not being able to grab data as easily from dynamic pages. In those situations we will use Selenium.

So, in the cell below we can start with a quick example. Let's try with a few pages to get a feel for how things change. So, below lets define three variables (url_1, url_2, and url_3) to three different urls.

The first is Zara's dresses page, flatiron data science page, and pelton's yelp page.

In [3]:
url_1 = 'https://www.zara.com/us/en/woman-dresses-l1066.html?v1=1180427'

So, first we need to use the Requests library to request the url and assign that to the variable page. We can then use page in the beautiful soup function, and pass 'lxml' as the second argument. This just means that Beautiful Soup is parcing the page as lxml.

By convention we will save the results of BeautifulSoup as soup (in this case FI_soup).

In [3]:
zara_page = requests.get(url_1)
zara_soup = BeautifulSoup(zara_page.content, 'lxml')

You can already view this, simply run the cell below to view the page html: 

In [4]:
zara_soup #this may time out due to the page being too big

zara_soup.find('div') #this will return the first div on the page

<div class="_global-loader active" id="global-loader"></div>

So, now that we have the html saved as a variable we can parce through the page to get the information we need.

In the pervious cell we saw one of the most common ways to grab information using Beautiful Soup. The two methods __find__ and __findAll__ will be your best friends in this process. However, there are many other ways to grab infomation from the html if these do not work. And we will discuss those next.

If we go and inspect our page, we can see the classnames and IDs of items that we want. below is the screen shot of what the it looked like when I inspected each product item. In this exercise, we will be grabbing the products name and price, and saving it as a dictionary.

<img style="float: center;" src="images/inspect_zara.png"  width=800>

if you look at it we can easily see the classname for the divs surrounding the product name and the price, **'name _item'** and **'price _product_price'** respectively.

#### Lets first see if we can grab all of the product names.

In Beautiful Soup the method findAll allows us to do just that. If we use the find method, it will only retun the first itme that it finds that meets the requirements passed, while finaAll returns every item that meets the requirements.

As seen above, the first thing we pass in the method is the html tag (i.e. 'div','p','h1','a') and the second will be a dictionary with a key value pair. Typically you will see **'class'** with the actual class name or **'ID'**. We will be using class.

After grabbing all of the links with this class name _'name'_ we will use a [list comprehension](https://hackernoon.com/list-comprehension-in-python-8895a785550b) to create a list of the names. If you are not familiar with list comprehensions, I highly recommend them! 

After, lets view the first 5 elements of the list:

In [5]:
names = zara_soup.findAll('a',{'class','name'})
product_names = [item.text for item in names]
product_names[:5]

['DRESS WITH DRAPED NECKLINE',
 'POLKA DOT DRESS',
 'LINEN DRESS WITH BUTTONS',
 'FRINGED TEXTURED WEAVE TUNIC',
 'RUFFLED DRESS']

The next part for prices gets a little complicated. Due to how the html is parced we need to use a little regex to grab the prices! So first, what we need to do is to grab the divs with the prices and lets look at the first item in the list before we move forward:

In [10]:
prices = zara_soup.findAll('div',{'class','_product-price'})
prices[0]

<div class="price _product-price"><span data-price="49.90 USD"></span></div>

Very frequently we can just use the dot method text, which would return the text in the divs. However, usually if the text is surrounded by a span we are unable to do this. If we test it below we can see that this returns an empty string:

In [158]:
# zara_soup.h1.descendants
for string in zara_soup.h1.children:
    print(repr(string))
#     print(i

"WOMEN'S DRESSES FOR EVERY MOMENT"


In [100]:
zara_soup.find('a',{'class','name'})

'DRESS WITH DRAPED NECKLINE'

Now for some Regex. We will use Python's 'Re' library's search method as well as it's group. If we look at all of the prices we realize they are surrounded in a span and have a special html tag called 'data-price'. So to grab this information pass the first chuck of html before the price <img style="display: inline-block;" src="images/span_img.png"  width=150> and before each special character we will use a '\' to break and let the regex parcer know that we need that special charater in our search. next we will have parentheses with '.*' in it which just means any length/combination of characters, and then finish it with ' USD"><' this is how our sting should end.

In [7]:
prod_prices = [float(re.search('\<span data\-price\=\"(.*)\ USD\"\>\<',str(item)).group(1)) for item in prices]
prod_prices[:5]

[49.9, 69.9, 69.9, 49.9, 49.9]

## Next - Selenium:

#### The majority of your projects will consist of these steps.

However, as i mentioned before there are other ways to grab data other than using the find and findAll dot methods for html tags. [This](https://selenium-python.readthedocs.io/locating-elements.html) is a great resource to hopefully give you some information on your different options, however I will outline them here.

Each of the following methods to locate items on the page follow the same two patterns. To **locate the first element** on the page that matches we do: **".find\_element\_by\_[name of item we want to find]"**. If we want to **find all of the items** and return them as a list we do: **".find\_elements\_by\_[name of item we want to find]"**. The items that can be used here are:
- id
- name
- xpath
- link_text
- partial_link_text
- tag_name
- class_name
- css_selector

These can replace the findAll and find methods from before, however I tend to keep the those methods and do not use id, name, tag_name, and class_name. The methods that have been most useful are xpath, link_text, and partial_link_text. I will walk through these three below.

But first, we need to expand on the last section and show where selenium can be really useful.

### Using Selenium to get the right data:

One issue I've run into multiple times is having many items on the page that are not what I need. So, you can filter out your results and then use this new url. But what if the url doesn't change? What if the data that you need can only be cleaned up by directly filtering you results while you're scraping? This is where Selenium comes in. Zara is a good example of this, but happens on many pages. 

Below lets look into this and how to filter the page properly. If you haven't already imported Selenium and the webdriver manager do this now. The ChromeDriver manager is good if you do not know where your chromedriver was installed. However, after you run this once, it will print where it found your driver--for whichever one you have. If you need more information on how to install selenium and the webdriver look [here for ChromeDriver](http://chromedriver.chromium.org/) and here for [WebDriver_Manager](https://github.com/SergeyPirogov/webdriver_manager). 

Lets create start our webdriver:

In [4]:
#create the chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install()) 
driver.get(url_1)



Checking for mac64 chromedriver:75.0.3770.90 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/75.0.3770.90/mac64/chromedriver


Once you run the WebDriverManager for the first time you will get an output that looks like: **Driver found in /Users/YOUR_USER_NAME/.wdm/chromedriver/75.0.3770.90/mac64/chromedriver**. You will then be able to put that into the webdriver.Chrome() parenthesis instead of the manager itself!

Now you should have seen a new chrome window open up to the url you specified. In my case, it'll be Zara's dresses page. Now, we need to inspect the filters to determine what we need to select:

<img style="float: center;" src="images/zara_filter_inspect.png"  width=600>

So, after inspecting the filter button, we can see that the class we need to be selecting by is: **'products-filter'**. Lets do this below:

_**NOTE**: At this point it may be tempting to select by partial link text, but this is not a linked item, even if it seems like it is._ 

In [5]:
#now, filter the page by the types of bags we want to see
driver.find_element_by_class_name('products-filter').click()
time.sleep(0.5)

__Important: you need to add sleep to your webdriver, and to any scraping scripts. It is not only beneficial to you, but also to the website owner. It will decrease your odds of being kicked off.__

You will now see the filter menu open up in your driver window. Next lets look into the different filters and select the items we need. The first thing to note is that each filter category is long enough that it is truncated to fit on the screen. So, first we will need to expand all of those. This is what it looks like when you inspect it:

<img style="float: center;" src="images/filter_inspect.png"  width=600>

In this case we will find all the elements with the class name **'\_view-more-button'**, and then click them. You can accomplish this by creating a list of

In [10]:
filter_sections = driver.find_elements_by_class_name('_view-more-button')
for item in filter_sections:
    time.sleep(0.5)
    item.click()
    

### .find\_element\_by\_xpath & .find\_elements\_by\_xpath

Xpath can be very useful. I have used it most when divs and other html elements do not provide a class_name or a class_name is the same as the other items. Sometimes, you only want to select the second item of a specific class and instead of grabbing all of them you can also use this.

There are other options to do the latter task and that will be briefly desribed next.

### Grab elements using their _siblings_

previously we discussed grabbing the next element in a list, or the next item that would be captured if you did the findAll method for a specific, or not so specific tag. Using the two methods .next_sibling and .previous_sibling can be very useful. 

Here is a quick example on how to implement both:

### .find\_element\_by\_link_text & .find\_elements\_by\_link_text

I will only go over by\_link\_text. These two methods are essentially the same. The biggest reason to use partial\_link\_text over \_link\_text is a simple one. Many times parts of a link or item on the page will remain the same, so if this is the case for you and you want to grab all of the items with similar text, then you would use only partial. However, if you have a very specific item you need then you would use all of it.